### Decision Tree

- The decision tree algorithm is a supervised learning algorithm -- we first construct the tree with historical data, and then use it to predict an outcome
- One of the major advantages of decision trees is that they can pick up nonlinear interactions between variables in the data that linear regression can't
- We can use trees for classification or regression problems.
- With larger dataset - say 100000 rows and 20 features, its difficult to build a decision tree manually by considering all the different possibilities. This is where the decision tree machine learning algorithm can help. It enables us to automatically construct a decision tree that tells us what outcomes we should predict in certain situations.
- A decision tree is made up of a series of nodes and branches. A node is where we split the data based on a variable, and a branch is one side of the split. 

### Entropy and Information Gain(IG)

- Entropy is the number of bits required to decode a given information
- Information Gain(IG) measues how much information a feature gives us about a class
- Decision Trees always try to maximize the IG
- An attribute with the highest IG will be split first

IG = entropy(parent/label) - [weighted avg] * entropy(children/feature)

$ IG(T,A) = Entropy(T) - \sum_{v \in A} \frac{|T_v|}{|T|}.Entropy(T_v)$

where <br>
 - T = parent
 - A = children

In [68]:
import scipy.stats as st
# no of events = 2
# probability of event 1 = 7/8
# probability of event 2 = 1/8
st.entropy([7,1],base=2)

0.5435644431995964

In [61]:
-(1/8*np.log2(1/8))*8

3.0

In [66]:
-((7/8)*(np.log2(7/8)) + (1/8)*np.log2(1/8))

0.5435644431995964

### Task

- Dataset: income.csv from 1994 census
- To predict: whether individuals make less than or equal to 50k a year, or more than 50k a year

### Categorical Data type

One strategy is to convert the columns to a categorical type. Under this approach, pandas will display the labels as strings, but internally store them as numbers so we can do computations with them

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
income = pd.read_csv("income.csv")


In [35]:
orig = pd.read_csv("income.csv")

In [36]:
orig.workclass.unique()

array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay',
       ' Never-worked'], dtype=object)

In [4]:
income.shape

(32561, 15)

In [5]:
income.head(5)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
income.high_income.unique()

array([' <=50K', ' >50K'], dtype=object)

In [8]:
income.workclass.head(5)

0            State-gov
1     Self-emp-not-inc
2              Private
3              Private
4              Private
Name: workclass, dtype: object

In [9]:
col = pd.Categorical(income.workclass)
income.workclass = col.codes
income.workclass.head(5)

0    7
1    6
2    4
3    4
4    4
Name: workclass, dtype: int8

In [69]:
high_income_c = pd.Categorical(income.high_income)

In [72]:
income['high_income_c'] = high_income_c.codes

In [118]:
np.bincount(income.workclass)

array([ 1836,   960,  2093,     7, 22696,  1116,  2541,  1298,    14],
      dtype=int64)

In [136]:
def split_on_median(val, median):
    if val <= median:
        return 0
    else:
        return 1
    
def information_gain(df, p, c,base=2,verbose=False):
    info_gain = 0.0
    parent = df[p]
    child = df[c]
    entropy_p = st.entropy(parent.values, base=base)
    entropy_c = 0.0
    median = np.median(child)
    split_child = "split_"+c
    df[split_child] = df[c].apply(lambda v: split_on_median(v, median))
    
    # grouped_children will have 2 groups
    grouped_children = df[split_child].value_counts().sort_index()
    child_len = len(child)
    for g in grouped_children.index:
        group = df[(df[split_child] == g)][p].values
        #print(group)
        entropy = st.entropy(group, base=base)
        entropy_c += (grouped_children[g]/child_len)* entropy
    
    info_gain = entropy_p - entropy_c
    
    if verbose: 
        print(f"Parent Entropy = {entropy_p}")
        print(f"Child Entropy = {entropy_c}")
        print(f"Info Gain = {info_gain}")
    return info_gain
        

In [141]:
for column in ["education", "marital_status", "occupation", "relationship", "race", "sex", "native_country"]:
    print(f"IF for {column} : {information_gain(income,'high_income',column)}")
#print(information_gain(income,"marital_status","workclass"))
#print(information_gain(income,"education","workclass"))
#print(information_gain(income,"occupation","workclass"))

IF for education : 0.8870260124881195
IF for marital_status : 1.502314710205022
IF for occupation : 0.9953955450472289
IF for relationship : 1.1169880934840588
IF for race : 0.0
IF for sex : 0.0
IF for native_country : 0.026178642839569832


In [145]:
def find_best_col(columns, target_column, df,verbose=False):
    max_ig = 0.0
    best_col = ""
    for col in columns:
        ig = information_gain(df, target_column, col)
        if ig > max_ig:
            max_ig = ig
            best_col = col
        if verbose:
            print(f"{col}: {ig}")
    return best_col

In [147]:
columns = ["education", "marital_status", "occupation", "relationship", "race", "sex", "native_country","hours_per_week"]
print(find_best_col(columns,"high_income", income,True))

education: 0.8870260124881195
marital_status: 1.502314710205022
occupation: 0.9953955450472289
relationship: 1.1169880934840588
race: 0.0
sex: 0.0
native_country: 0.026178642839569832
hours_per_week: 0.9904272497830089
marital_status


In [80]:
st.entropy(income.high_income.values,base=2)

12.936821944492381

In [14]:
for name in ["education", "marital_status", "occupation", "relationship", "race", "sex", "native_country", "high_income"]:
    col = pd.Categorical(income[name])
    income[name] = col.codes